# HTML解析入门及准备URL生成连续技

*  本周主要内容：批量抓取页面基础及技巧
*  上周主要内容：HTML解析（parse HTML）及准备URL生成连续技
*  20春_Web数据挖掘_week04
*  电子讲义设计者：廖汉腾, 许智超
<br/>
<br/>

-----
## 复习

复习：上周内容，实践

* 翻页：参数字典的拆解
* 翻页：系统性迭代
* 翻页：数据备份与整合

-----
## 本周内容及学习目标

搜狗微信：搜索关键字获取微信公众号相关文章
  
### 目标
1. 使用 requests-html 爬取并存取网页文字档，查找[requests-html 中文文档](https://cncert.github.io/requests-html-doc-cn/#/)
2. 熟悉 [xpath 语法](https://www.w3cschool.cn/xpath/xpath-syntax.html)丶[xpath 节点](https://www.w3cschool.cn/xpath/xpath-nodes.html)
3. 使用 [xpath cheatsheet](https://devhints.io/xpath)
  * 在 Chrome Inspector 使用
  * 在 requests-html (Python) 使用
4. 简易使用 [pd.DataFrame](https://www.pypandas.cn/doc/getting_started/dsintro.html#dataframe)
5. 参数字典的拆解与迭代
6. 翻页数据备份与整合

In [2]:
%%html
<style>
/* 本电子讲义使用之CSS */
div.code_cell {
    background-color: #e5f1fe;
}
div.cell.selected {
    background-color: #effee2;
    font-size: 2rem;
    line-height: 2.4rem;
}
div.cell.selected .rendered_html table {
    font-size: 2rem !important;
    line-height: 2.4rem !important;
}
.rendered_html pre code {
    background-color: #C4E4ff;   
    padding: 2px 25px;
}
.rendered_html pre {
    background-color: #99c9ff;
}
div.code_cell .CodeMirror {
    font-size: 2rem !important;
    line-height: 2.4rem !important;
}
.rendered_html img, .rendered_html svg {
    max-width: 60%;
    height: auto;
    float: right;
}

.rendered_html img[src*="#full"], .rendered_html svg[src*="#full"] {
    max-width: 100%;
    height: auto;
    float: none;
}

.rendered_html img[src*="#thumbnail"], .rendered_html svg[src*="#thumbnail"] {
    max-width: 15%;
    height: auto;
}

/* Gradient transparent - color - transparent */
hr {
    border: 0;
    border-bottom: 1px dashed #ccc;
}
.emoticon{
    font-size: 5rem;
    line-height: 4.4rem;
    text-align: center;
    vertical-align: middle;
}
.bg-split_apply_comine {
    width: 500px;     
    height: 300px;
    background: url('02_split-apply-comine_500x300.png') -10px -10px;
    float: right;
}
.bg-comine {
    width: 175px;
    height: 150px;
    background: url('02_split-apply-comine_500x300.png') -280px -80px;
    float: right;
}
.bg-apply {
    width: 155px;
    height: 225px;
    background: url('02_split-apply-comine_500x300.png') -160px -30px;
    float: right;
}
.bg-split {
    width: 205px;
    height: 225px;
    background: url('02_split-apply-comine_500x300.png') -10px -30px;
    float: right;
}
.break {
                   page-break-after: right; 
                   width:700px;
                   clear:both;
}
</style>

In [330]:
# 基本模块
import pandas as pd
from requests_html import HTMLSession
from pandas.api.types import is_numeric_dtype

# 搜狗微信：文章搜索(模仿liepin)

In [533]:
# A-1 query 为关键字选取 以python为例

url = "https://weixin.sogou.com/weixin"
session = HTMLSession()
payload = {
    "query":"UX",
    "type":"2", # 2 文章、1 公众号
    "page":10  
}

r = session.get( url, params = payload, )


主要元素 =  r.html.xpath( '//ul[@class="news-list"]/li')

dict_xpaths = {
    'text':{
        'source':'//div[contains(@class,"txt-box")]/div[@class="s-p"]/a/text()',
        '时间':'//div[contains(@class,"txt-box")]/div[@class="s-p"]/@t',       
    },
    'text_content':{
        '标题':'//div[contains(@class,"txt-box")]/h3/a',
        '摘要':'//div[contains(@class,"txt-box")]/p'
    },
    'href':{
        "公众号链接":'//div[contains(@class,"txt-box")]/div[@class="s-p"]/a/@href',
        "文章链接":'//div[contains(@class,"txt-box")]/h3/a/@href',
    }
}


def get_e_text_content(_xpath_):
    # 高级列表推导
    暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]

    return(暂存结果)

def get_e_text(_xpath_):
    暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
    return(暂存结果)

def get_e_href(_xpath_):
    # 高级列表推导
    暂存结果 = ["https://weixin.sogou.com"+item\
            for item in e.xpath(_xpath_)\
            for e in 主要元素]
    return(暂存结果)

数据字典 = dict()
# content
数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}
# text
数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
# href
数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

数据 = pd.DataFrame(数据字典)
数据 = 数据.assign(
    格式化时间 = lambda x: pd.to_datetime(x["时间"], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
)
数据



,标题,摘要,source,时间,公众号链接,文章链接,格式化时间
0,2020年UI/UX九大流行趋势,,互联网架构师,1585701057,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-01 00:30:57
1,#不E样的心机体验#焕新UX设计体验如何?话筒给他们!,"今天,我们一起来看看花粉朋友们对于EMUI10“美,不止初见”的UX设计有哪些使用感受吧!焕...",华为EMUI,1582106885,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-02-19 10:08:05
2,手机增强现实应用程序的UX设计规范,"为了确保未来MAR系统被认可和成功,也为了填补MAR研究的空白,确定适用于MAR应用的当下U...",MicroUX,1583817775,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-03-10 05:22:55
3,全新雷克萨斯纯电动UX 300e 开启预热,"全新雷克萨斯纯电动UX 300e于广州车展全球首发,经过几个月的等待,即日起全面开启预热.欢...",临沂和凌雷克萨斯4S店,1586830340,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 02:12:20
4,2020年UI/UX九大流行趋势,,前端达人,1584765310,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-03-21 04:35:10
5,"选口碑豪华SUV,XT4、冒险家、UX的优势在哪里?","UX售价28.30万元起,假如它能再多多提供终端优惠,我相信它也能和加价才能提车的兄弟ES一...",车闻社,1585218357,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-03-26 10:25:57
6,【视频】2020 UX设计趋势,"“用户体验”是过去十年的流行语之一,在过去的几年中,UX已经从设计世界的另一个口号变成了世界...",设计帝国网站,1585702828,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-01 01:00:28
7,UX.LAB实验室-防疫设计大赛开始了!,"UX.LAB实验室防疫设计大赛开始了!(防疫设计主题作品示例,素材来源自花瓣网)01活动目的...",用户体验实验室,1585717688,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-01 05:08:08
8,2020年9大 UX设计趋势,,美啊教育,1585648837,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-03-31 10:00:37
9,雷克萨斯加速电气化战略 首款纯电动车UX 300e年内进入中国及欧洲市场 | 经观汽车,"王海宣受新冠肺炎疫情影响,日内瓦车展被迫取消,而雷克萨斯原本在日内瓦车展亮相的旗下首款纯电动...",经观汽车,1583476788,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-03-06 06:39:48


In [531]:
with open ("weixin_UX.html", encoding = "utf8", mode = "w") as fp:
    fp.write(r.html.html)

## xpath的具体操作

In [493]:
# 来源的xpath解析结果
r.html.xpath('//div[contains(@class,"txt-box")]/div[@class="s-p"]/a/text()')

['CSDN',
 'CSDN',
 'Python数据之道',
 '大学生必备网',
 '寻梦暖屋',
 'Python程序员',
 'InfoQ',
 'Python那些事',
 'Python小屋',
 'Python程序员']

In [495]:
# 标题？1、em text()? + a/text()?
# 某一条<li>信息,再进行上一步 if else
r.html.xpath('//div[contains(@class,"txt-box")]/h3/a/text()')


[' 十大装 B 语法',
 '势头强劲的 ',
 ' PK 强大的 C++,究竟谁更胜一筹?',
 '很冷门,用起来却非常实用',
 '库',
 '9.9元',
 '小课体验课,限100人!',
 '今日学习-',
 '集合(1)',
 '找对象那么难?',
 ' s3cmd 帮你解决',
 '为什么 ',
 ' 不是未来的编程语言?',
 '7 款 ',
 ' 开源框架的优劣总结',
 '绘制傅里叶变换、反变换与带通滤波图像',
 '用',
 '模拟新冠肺炎的传播']

In [500]:
r.html.xpath('//div[contains(@class,"txt-box")]/h3/a')[0].lxml.text_content()

'Python 十大装 B 语法'

In [502]:
r.html.xpath('//div[contains(@class,"txt-box")]/p')[0].lxml.text_content()

'作者 | 许向武责编 | 郭芮出品 | CSDN 博客Python 是一种代表简单思想的语言,其语法相对简单,很容易上手.不过,如果就此小视 ...'

In [503]:
# 公众号链接
'https://weixin.sogou.com/weixin'+r.html.xpath('//div[contains(@class,"txt-box")]/div[@class="s-p"]/a/@href')

['/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6EzDJysI4ql5MPrOUp16838dGRMI7NnPq5hmM2JyHRhS2xrPG4asQ7AwvDqyjOWdzeLxXplXAcuNuVuNPiFqscsKWDWntqkEi14zy27r5NbdFls1_OnGoFix4ZDKkBJOx-2Ls84MFeMX_AicIp_zOvFbsuFhbVQ2Rthlp8MQWpUw2Ae8Afa8duOO00efWrWmm&type=2&query=python&token=18C534345BEFB54D9F9A3C97CC9C9383A01FE1A55E9525FE',
 '/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6EzDJysI4ql5MPrOUp16838dGRMI7NnPq5hmM2JyHRhS2xrPG4asQ7AwvDqyjOWdzeLxXplXAcuNuVuNPiFqscsKWDWntqkEi14zy27r5NbdFls1_OnGoFix4ZDKkBJOx-2Ls84MFeMX_AicIp_zOvFbsuFhbVQ2Rthlp8MQWpUw2Ae8Afa8duOO00efWrWmm&type=2&query=python&token=18C534345BEFB54D9F9A3C97CC9C9383A01FE1A55E9525FE',
 '/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6EzDJysI4ql5MPrOUp16838dGRMI7NnPq5hmM2JyHRhS2xrPG4asQ7AwvDqyjOWdz0w9ljRZpbI9M_GboL8uxYw0aK1zsA837eBDt0gCrk0YQB5oodUcHb3fERD_b9RnkhAyc1Bu02V_mR1_SalCO2XhqiE_LS1nxTzOB9g_EDYCxvSF_admVdGRTpLLTNgHY&type=2&query=python&token=18C534345BEFB54D9F9A3C97CC9C9383A01FE1A55E9525FE',
 '/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6EzDJysI4ql5MPr

In [ ]:
# 公众号文章链接

In [ ]:
# 时间

In [ ]:
# 图片

In [489]:
r.request.headers

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/603.3.8 (KHTML, like Gecko) Version/10.1.2 Safari/603.3.8', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Cookie': 'IPLOC=CN4419; SNUID=AEDFDD61181CBA2A31789BFB19535630; SUID=B6C6C5784A42910A000000005E94CF61; ABTEST=8|1586810721|v1'}

In [486]:
# A-2 下载页面
with open ("weixin_用户研究.html", encoding = "utf8", mode = "w") as fp:
    fp.write(r.html.html)

In [444]:
# lxml.text_content()
r.html.xpath('//div[contains(@class,"txt-box")]/h3/a/em/text()')
r.html.xpath('//div[contains(@class,"txt-box")]/h3/a/text()')
r.html.xpath('//div[contains(@class,"txt-box")]/p')[0].lxml.text_content()

'1.库引用与import函数2.库引用:扩充Python程序功能的方式3.使用import保留字,采用. ()的风格4.import    . (函数参数)5.使用保...'

In [446]:
r.html.xpath('//div[contains(@class,"txt-box")]/h3/a/@href')
r.html.xpath('//div[contains(@class,"txt-box")]/div[@class="s-p"]/a/@href')

['/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6EzDJysI4ql5MPrOUp16838dGRMI7NnPqDMsPE5ZFEm9svg4Skk4qHgwvDqyjOWdzqJJllMV4sgV_iqjAviJMH-nun9V5cWucQzQ0H3lPk1SaWMC6z7zW3ZuD2Fblyv0UPqDp0JAtd4YsPXStQJx02ZcC0X5jBZq0XqaPxPMr9WBrN6ixXvK44WRTpLLTNgHY&type=2&query=python&token=161181389B20209DE6E046D7D1CD6454E666492D5E94BF4F',
 '/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6EzDJysI4ql5MPrOUp16838dGRMI7NnPqDMsPE5ZFEm9svg4Skk4qHgwvDqyjOWdzM1N3sg6-MGMs4Ovzv0sOdV6iNSk94nbViiSSJhj5RDOB07es2XIcPyyXi_McJDQhaXt1_F979mkpTPofR_EpXMdI9FUlMBCcnD1ESuzbUbKvPER_fgAHB-O00efWrWmm&type=2&query=python&token=161181389B20209DE6E046D7D1CD6454E666492D5E94BF4F',
 '/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6EzDJysI4ql5MPrOUp16838dGRMI7NnPqDMsPE5ZFEm9svg4Skk4qHgwvDqyjOWdzM1N3sg6-MGMs4Ovzv0sOdV6iNSk94nbViiSSJhj5RDOB07es2XIcPyyXi_McJDQhaXt1_F979mkpTPofR_EpXMdI9FUlMBCcnD1ESuzbUbKvPER_fgAHB-O00efWrWmm&type=2&query=python&token=161181389B20209DE6E046D7D1CD6454E666492D5E94BF4F',
 '/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6EzDJysI4ql5MPr

In [409]:
r.request.headers

{'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_12_6) AppleWebKit/603.3.8 (KHTML, like Gecko) Version/10.1.2 Safari/603.3.8', 'Accept-Encoding': 'gzip, deflate', 'Accept': '*/*', 'Connection': 'keep-alive', 'Cookie': 'ABTEST=0|1586803293|v1'}

## 下载页面和输出excel

In [405]:
# A-2 下载页面
with open ("weixin_python.html", encoding = "utf8", mode = "w") as fp:
    fp.write(r.html.html)

In [337]:
# A-3 输出excel档
数据.to_excel("20春_Web数据挖掘_week05_sougou_weixin.xlsx", sheet_name="搜查结果")

/Users/xuzhichao/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:910: UserWarning: Ignoring URL 'http://weixin.sogou.com/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6EzDJysI4ql5MPrOUp16838dGRMI7NnPqR9qmSowG7r2gBNPyrh3FVgwvDqyjOWdzeOCKnRJAxkUdZw5uyu3Hynw3YU8OWoUNYcaOf58YCgqXH6IVD2Cttu-1hfDLPAhT5AG8baJOzSK67Mho2fdeLHDctAPZVNzluOsufU76d3ec5RqUeByHO-O00efWrWmm&type=2&query=python&token=210ED24AF7431EDE0C0EA8225BA57DCD0C67D9EF5E948ED8' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/Users/xuzhichao/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:910: UserWarning: Ignoring URL 'http://weixin.sogou.com/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6cwJThYulHEtVjXrGTiVgS9dZesVPC425bopQRjCb2BlpQ4aYX0iWk1qXa8Fplpd9Hhup-WCiA6jKWLk-8yqbp2an3quTv2SP3gDmUfMBWk4hAl2leQHjMIc7a1X13FuuMHylJLJweSgqKYEhdkYntOId8_j3Lu1YmkUd6oG33m7wXpIptlAt7U6T7RdGxQgLBllaLKbmvInP_dLDClI7biNRhieJZTrlVKy4Eeemcj_f3q8uRDTfkw..&type=2&query=python&to

## 拆分

### 文章链接

In [455]:
# A-4 文章链接 细化
# href_list 文章链接
article_href_list = []
for item in r.html.xpath('//div[contains(@class,"txt-box")]/h3/a/@href'):
#     print(item)
    article_href_list.append("https://weixin.sogou.com"+item)
article_href_list

['https://weixin.sogou.com/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6cwJThYulHEtVjXrGTiVgSwsS8AdK1eMIbKJrqUVRebxpQ4aYX0iWk1qXa8Fplpd97i_eAsv0X81DiwWew4H0AaNsMK1qfl8HkSWQqS3EiyzUFNg-DVu7GuC4PlMoY2SZ_ii9dfOFIMexihLz7kHYgkgX6LH4moKZc9L8tTe7P-EO50E8KbfZhLNJp3eVh_8iF-R_stCu8CFVtLsipj51Y1YKu-SkfyJu1UTGhkvmMcDI3OTRHYU2og..&type=2&query=python&token=161181389B20209DE6E046D7D1CD6454E666492D5E94BF4F',
 'https://weixin.sogou.com/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6cwJThYulHEtVjXrGTiVgSwsS8AdK1eMIbKJrqUVRebxpQ4aYX0iWk1qXa8Fplpd9zeBZZku4K1O1kyaw8CKLILyXbe7mAXT-bAqhSEEBCgV6IWHP1R-3BC0oQu1losxIZyC9SY5utNXbNHiAS20VK-qimX6x_JIA6N9FubBW4KLQGjyyZyQ-HWLYI2RFgvGygyFUtdMVsBY2NZgbJovqw3pvBWjJskQLxFw33riXFU_yPfCoem7FzA..&type=2&query=python&token=161181389B20209DE6E046D7D1CD6454E666492D5E94BF4F',
 'https://weixin.sogou.com/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6cwJThYulHEtVjXrGTiVgSwsS8AdK1eMIbKJrqUVRebxpQ4aYX0iWk1qXa8Fplpd9zeBZZku4K1O1kyaw8CKLILyXbe7mAXT-bAqhSEEBCgV6IWHP1R-3BAfW-gC2icbsb3fKwKK9nOLHSMVp

### 公众号链接

In [449]:
# A-5 公众号链接 细化
["http://weixin.sogou.com"+item\
 for item in e.xpath('//div[contains(@class,"txt-box")]/div[@class="s-p"]/a/@href')\
 for e in 主要元素]

['http://weixin.sogou.com/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6EzDJysI4ql5MPrOUp16838dGRMI7NnPqR9qmSowG7r2gBNPyrh3FVgwvDqyjOWdzeOCKnRJAxkUdZw5uyu3Hynw3YU8OWoUNYcaOf58YCgqXH6IVD2Cttu-1hfDLPAhT5AG8baJOzSK67Mho2fdeLHDctAPZVNzluOsufU76d3ec5RqUeByHO-O00efWrWmm&type=2&query=python&token=210ED24AF7431EDE0C0EA8225BA57DCD0C67D9EF5E948ED8',
 'http://weixin.sogou.com/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6EzDJysI4ql5MPrOUp16838dGRMI7NnPqR9qmSowG7r2gBNPyrh3FVgwvDqyjOWdzeOCKnRJAxkUdZw5uyu3Hynw3YU8OWoUNYcaOf58YCgqXH6IVD2Cttu-1hfDLPAhT5AG8baJOzSK67Mho2fdeLHDctAPZVNzluOsufU76d3ec5RqUeByHO-O00efWrWmm&type=2&query=python&token=210ED24AF7431EDE0C0EA8225BA57DCD0C67D9EF5E948ED8',
 'http://weixin.sogou.com/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6EzDJysI4ql5MPrOUp16838dGRMI7NnPqR9qmSowG7r2gBNPyrh3FVgwvDqyjOWdzeOCKnRJAxkUdZw5uyu3Hynw3YU8OWoUNYcaOf58YCgqXH6IVD2Cttu-1hfDLPAhT5AG8baJOzSK67Mho2fdeLHDctAPZVNzluOsufU76d3ec5RqUeByHO-O00efWrWmm&type=2&query=python&token=210ED24AF7431EDE0C0EA8225BA57DCD0C67D9EF

### 来源和发表时间

In [119]:
# A-6 来源和发表时间 细化
source = r.html.xpath('//div[contains(@class,"txt-box")]/div[@class="s-p"]/a/text()')
time =  r.html.xpath('//div[contains(@class,"txt-box")]/div[@class="s-p"]/@t')

# 翻页：参数字典的拆解

## xpath解析翻页a/@href

In [393]:
# B-0   单一页面
url = "https://weixin.sogou.com/weixin"
session = HTMLSession()
payload = {
    "query":"python",
    "type":"2"
}

r = session.get( url, params = payload)

In [509]:
# A-1  xpath 解析翻页a/@href
xpath_翻页a = '//div[@class="p-fy"]/a'
xpath_翻页a = '//div[@class="p-fy"]/a'
print (r.html.xpath(xpath_翻页a))

href_列表 = [x.xpath('//@href')[0] for x in r.html.xpath(xpath_翻页a)]
print (href_列表)

href_字典 = {x.text:x.xpath('//@href')[0]  for x in r.html.xpath(xpath_翻页a)}
print (href_字典)

[<Element 'a' href='?query=python&type=2&page=2&ie=utf8' id='sogou_page_2' uigs='page_2'>, <Element 'a' href='?query=python&type=2&page=3&ie=utf8' id='sogou_page_3' uigs='page_3'>, <Element 'a' href='?query=python&type=2&page=4&ie=utf8' id='sogou_page_4' uigs='page_4'>, <Element 'a' href='?query=python&type=2&page=5&ie=utf8' id='sogou_page_5' uigs='page_5'>, <Element 'a' href='?query=python&type=2&page=6&ie=utf8' id='sogou_page_6' uigs='page_6'>, <Element 'a' href='?query=python&type=2&page=7&ie=utf8' id='sogou_page_7' uigs='page_7'>, <Element 'a' href='?query=python&type=2&page=8&ie=utf8' id='sogou_page_8' uigs='page_8'>, <Element 'a' href='?query=python&type=2&page=9&ie=utf8' id='sogou_page_9' uigs='page_9'>, <Element 'a' href='?query=python&type=2&page=10&ie=utf8' id='sogou_page_10' uigs='page_10'>, <Element 'a' class=('np',) href='?query=python&type=2&page=2&ie=utf8' id='sogou_next' uigs='page_next'>]
['?query=python&type=2&page=2&ie=utf8', '?query=python&type=2&page=3&ie=utf8', '?

In [511]:
parse_qs("query=python&type=2&page=2&ie=utf8")

{'query': ['python'], 'type': ['2'], 'page': ['2'], 'ie': ['utf8']}

In [395]:
# A-2 建构参数模板：找到关键参数及参数结构

# 需要模组库
from urllib.parse import urlparse, parse_qs
import pandas as pd
from IPython.display import display, HTML



df = pd.DataFrame([ urlparse(x) for x in href_列表])

df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query'] ])

display(df)



print(df.nunique())
display(df_qs)
print(df_qs.nunique())

df_qs.page
df_qs = df_qs.assign (page_int=df_qs.page.astype(int)) # 变成整数(数值)
display(df_qs)

,scheme,netloc,path,params,query,fragment
0,,,,,query=python&type=2&page=2&ie=utf8,
1,,,,,query=python&type=2&page=3&ie=utf8,
2,,,,,query=python&type=2&page=4&ie=utf8,
3,,,,,query=python&type=2&page=5&ie=utf8,
4,,,,,query=python&type=2&page=6&ie=utf8,
5,,,,,query=python&type=2&page=7&ie=utf8,
6,,,,,query=python&type=2&page=8&ie=utf8,
7,,,,,query=python&type=2&page=9&ie=utf8,
8,,,,,query=python&type=2&page=10&ie=utf8,
9,,,,,query=python&type=2&page=2&ie=utf8,


scheme      1
netloc      1
path        1
params      1
query       9
fragment    1
dtype: int64


,query,type,page,ie
0,python,2,2,utf8
1,python,2,3,utf8
2,python,2,4,utf8
3,python,2,5,utf8
4,python,2,6,utf8
5,python,2,7,utf8
6,python,2,8,utf8
7,python,2,9,utf8
8,python,2,10,utf8
9,python,2,2,utf8


query    1
type     1
page     9
ie       1
dtype: int64


,query,type,page,ie,page_int
0,python,2,2,utf8,2
1,python,2,3,utf8,3
2,python,2,4,utf8,4
3,python,2,5,utf8,5
4,python,2,6,utf8,6
5,python,2,7,utf8,7
6,python,2,8,utf8,8
7,python,2,9,utf8,9
8,python,2,10,utf8,10
9,python,2,2,utf8,2


In [512]:
# A-2 建构参数模板：找到关键参数及参数结构

def parse_url_qs_for_page (url):
    six_parts = urlparse(url) 
    out = parse_qs(six_parts.query)
    return (out)

# 取一例做模板
参数模板 = parse_url_qs_for_page(href_列表[2])
print (参数模板)

# 重新审视：href_字典
print (href_字典)

{'query': ['python'], 'type': ['2'], 'page': ['4'], 'ie': ['utf8']}
{'2': '?query=python&type=2&page=2&ie=utf8', '3': '?query=python&type=2&page=3&ie=utf8', '4': '?query=python&type=2&page=4&ie=utf8', '5': '?query=python&type=2&page=5&ie=utf8', '6': '?query=python&type=2&page=6&ie=utf8', '7': '?query=python&type=2&page=7&ie=utf8', '8': '?query=python&type=2&page=8&ie=utf8', '9': '?query=python&type=2&page=9&ie=utf8', '10': '?query=python&type=2&page=10&ie=utf8', '下一页': '?query=python&type=2&page=2&ie=utf8'}


In [513]:
# A-3 建构参数模板生成器：query（搜索关键字）page
def 参数模板生成(query, page):
    参数 = 参数模板.copy()
    参数['page'] = page
    参数['query'] = query
    return (参数)

# -----------    参数模版生成器 目的 构建payload
# payload={
    
# }
参数_query_用户体验_page = { 
    i:参数模板生成(page = [i], \
                  query = ['用户体验']) \
    for i,v in href_字典.items()\
    }

print (df_qs.page_int.min()) # 最小值只有2 1的时候没有链接？
print (df_qs.page_int.max()) # 最大值只有9

参数_query_用户体验_page = { 
    i:参数模板生成(page = [i], \
                  query = ['用户体验']) \
    for i in range(df_qs.page_int.min()-1,df_qs.page_int.max()+1)\
    }
参数_query_用户体验_page

2
10


{1: {'query': ['用户体验'], 'type': ['2'], 'page': [1], 'ie': ['utf8']},
 2: {'query': ['用户体验'], 'type': ['2'], 'page': [2], 'ie': ['utf8']},
 3: {'query': ['用户体验'], 'type': ['2'], 'page': [3], 'ie': ['utf8']},
 4: {'query': ['用户体验'], 'type': ['2'], 'page': [4], 'ie': ['utf8']},
 5: {'query': ['用户体验'], 'type': ['2'], 'page': [5], 'ie': ['utf8']},
 6: {'query': ['用户体验'], 'type': ['2'], 'page': [6], 'ie': ['utf8']},
 7: {'query': ['用户体验'], 'type': ['2'], 'page': [7], 'ie': ['utf8']},
 8: {'query': ['用户体验'], 'type': ['2'], 'page': [8], 'ie': ['utf8']},
 9: {'query': ['用户体验'], 'type': ['2'], 'page': [9], 'ie': ['utf8']},
 10: {'query': ['用户体验'], 'type': ['2'], 'page': [10], 'ie': ['utf8']}}

# 封装request_sougou_weixin

In [534]:
session = HTMLSession()
def request_sougou_weixin(url,params):
   
    r = session.get( url, params = payload, )
    
    
    主要元素 =  r.html.xpath( '//ul[@class="news-list"]/li')

    dict_xpaths = {
        'text':{
            'source':'//div[contains(@class,"txt-box")]/div[@class="s-p"]/a/text()',
            '时间':'//div[contains(@class,"txt-box")]/div[@class="s-p"]/@t',       
        },
        'text_content':{
            '标题':'//div[contains(@class,"txt-box")]/h3/a',
            '摘要':'//div[contains(@class,"txt-box")]/p'
        },
        'href':{
            "公众号链接":'//div[contains(@class,"txt-box")]/div[@class="s-p"]/a/@href',
            "文章链接":'//div[contains(@class,"txt-box")]/h3/a/@href',
        }
    }


    def get_e_text_content(_xpath_):
        # 高级列表推导
        暂存结果 = [e.xpath(_xpath_)[0].lxml.text_content() for e in 主要元素]

        return(暂存结果)

    def get_e_text(_xpath_):
        暂存结果 = ["".join([x.strip() if type(x) is str else x.text.strip() for x in e.xpath(_xpath_)]) for e in 主要元素]
        return(暂存结果)

    def get_e_href(_xpath_):
        # 高级列表推导
        暂存结果 = ["https://weixin.sogou.com"+item\
                for item in e.xpath(_xpath_)\
                for e in 主要元素]
        return(暂存结果)

    数据字典 = dict()
    数据字典 = {k:get_e_text_content(v) for k,v in dict_xpaths['text_content'].items()}

    数据字典.update({k:get_e_text(v) for k,v in dict_xpaths['text'].items()})
    数据字典.update({k:get_e_href(v) for k,v in dict_xpaths['href'].items()})

    数据 = pd.DataFrame(数据字典)
    数据 = 数据.assign(
        格式化时间 = lambda x: pd.to_datetime(x["时间"], unit='s').dt.strftime('%Y-%m-%d %H:%M:%S')
    )
    return(数据)


url = "https://weixin.sogou.com/weixin"


list_df=[]
for i in range(1,10):
    print("第",i,"页")
    payload = {
        "query":"python",
        "type":"2",
        "page":i

    } 
    print("payload:",payload)
    print("url:",url)
    time.sleep(10+10*random())
    df = request_sougou_weixin(url,params=payload)
    print("第",i,"页","的Dataframe")
    display(df)
    
    df.to_csv("20春_Web数据挖掘_week05_sougou_weixin_{key}_{k}.tsv"\
                  .format(key="python", k=i), sep="\t", encoding="utf8")
        
    df = df.assign (query = key)  # 区分  query   
    df = df.assign (page = i)  # 区分  page    
    list_df.append(df)
    

df_all = pd.concat(list_df).reset_index()
df_all.index.name = '序'

df_all.to_excel("20春_Web数据挖掘_week05_sougou_weixin_翻页_5.xlsx",\
                sheet_name="_".join(query))
# 预估时间: 4*5秒*10 =200
# 预估数量: 4*40*10 =1600

第 1 页
payload: {'query': 'python', 'type': '2', 'page': 1}
url: https://weixin.sogou.com/weixin
第 1 页 的Dataframe


,标题,摘要,source,时间,公众号链接,文章链接,格式化时间
0,手把手教你用 Python 绘制酷炫的桑基图!,"Python手把手绘制桑基图动手之前,我们再次敲黑板,回顾桑基图组成要素的重点——节点、边和...",CSDN,1585706777,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-01 02:06:17
1,Python 十大装 B 语法,作者 | 许向武责编 | 郭芮出品 | CSDN 博客Python 是一种代表简单思想的语言...,CSDN,1572764522,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2019-11-03 07:02:02
2,找对象那么难?python s3cmd 帮你解决,"python 的第三方库的安装目录:这个库实现 S3 协议的封装,解析,发包,收包等操作命令...",Python程序员,1586823024,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 00:10:24
3,"9.9元Python小课体验课,限100人!","Python几乎正逐渐成为,未来5~10年新一代“Office技能”.01初入职场如何不被轻...",大学生必备网,1586827802,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 01:30:02
4,为什么 Python 不是未来的编程语言?,"作者丨Rhea Moutafis译者丨刘志勇策划丨刘燕Python 现在如此火爆,已发展成为...",InfoQ,1586824246,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 00:30:46
5,"小白怎样学python,才不会从入门到放弃?","Python已进入小学教材将加入高考编程可以将我们从重复的工作中解放出来,去完成更多更有创造...",业余无线电,1586826425,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 01:07:05
6,7 款 Python 开源框架的优劣总结,下面就简单的介绍一下Python七款开源框架的优劣.DjangoDjango 应该是最出名的...,Python那些事,1586823354,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 00:15:54
7,Python绘制傅里叶变换、反变换与带通滤波图像,"推荐图书:《Python可以这样学》,ISBN:9787302456469,董付国,清华大学...",Python小屋,1586818537,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-13 22:55:37
8,跟李老师学编程-Python3 错误和异常,"作为 Python 初学者,在刚学习 Python 编程时,经常会看到一些报错信息,在前面我...",极客青少编程,1586826032,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 01:00:32
9,用Python模拟新冠肺炎的传播,我想到的是单纯地在Python中使用可视化工具matplotlib来编写这个模拟过程.但是对...,Python程序员,1586823024,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 00:10:24


第 2 页
payload: {'query': 'python', 'type': '2', 'page': 2}
url: https://weixin.sogou.com/weixin
第 2 页 的Dataframe


,标题,摘要,source,时间,公众号链接,文章链接,格式化时间
0,MySQL与Python交互入门,"(0, ＂python1＂, 50), (0, ＂python2＂, 60), (0, ＂p...",小詹学Python,1586824285,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 00:31:25
1,Python到底是什么?学姐靠它拿了5个offer,“熟悉Python编程优先”这样的条件.需要的不是埋头苦干而是一次实现你需要的不是埋头苦干而...,遇见百种人生,1586790385,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-13 15:06:25
2,python进阶-函数式编程(高级函数),"python sorted(iterable, key=None, reverse=Fals...",StriveNow,1586799046,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-13 17:30:46
3,Python中 Pandas 50题冲关,Python中的Numpy基础20问参考资料 | 100-pandas-puzzles - ...,Python那些事,1586823354,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 00:15:54
4,Python内存管理机制,"在之前的面试中有被问到Python的内存管理机制,回答的不好,之后查阅了一些文章,这里简单做...",Begin Now,1586822446,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 00:00:46
5,Python小课教程:模块,"在Python中,一个.py文件就称之为一个模块(Module).使用模块有什么好处?最大的...",Python小课教程,1586793485,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-13 15:58:05
6,Python 碾压 Excel,"根据最新消息,Python成了各大公司的心头好投行、券商、咨询、四大纷纷表示:想要顺利求职,...",证经学社,1586825623,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 00:53:43
7,"势头强劲的 Python PK 强大的 C++,究竟谁更胜一筹?","1.如果在Windows机器上用Windows编译器来编译代码,则会生成Windows机器代...",CSDN,1580544534,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-02-01 08:08:54
8,Python 什么时候会被取代?,"为什么Python不是未来的编程语言?作者 | Rhea Moutafis译者 | 弯月,责...",CSDN,1586052093,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-05 02:01:33
9,没错!Python杀死了Excel!,Python数据分析有多强大?举个栗子:用几行代码就能快速整理数据并出图:甚至可以编写合成多...,CSDN,1583654301,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-03-08 07:58:21


第 3 页
payload: {'query': 'python', 'type': '2', 'page': 3}
url: https://weixin.sogou.com/weixin
第 3 页 的Dataframe


...

第 8 页
payload: {'query': 'python', 'type': '2', 'page': 8}
url: https://weixin.sogou.com/weixin
第 8 页 的Dataframe


,标题,摘要,source,时间,公众号链接,文章链接,格式化时间


第 9 页
payload: {'query': 'python', 'type': '2', 'page': 9}
url: https://weixin.sogou.com/weixin
第 9 页 的Dataframe


,标题,摘要,source,时间,公众号链接,文章链接,格式化时间


/Users/xuzhichao/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:910: UserWarning: Ignoring URL 'https://weixin.sogou.com/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6EzDJysI4ql5MPrOUp16838dGRMI7NnPqR9qmSowG7r2gBNPyrh3FVgwvDqyjOWdzeOCKnRJAxkUdZw5uyu3Hynw3YU8OWoUNYcaOf58YCgqXH6IVD2Cttu-1hfDLPAhT5AG8baJOzSK67Mho2fdeLHDctAPZVNzluOsufU76d3ec5RqUeByHO-O00efWrWmm&type=2&query=python&token=210ED24AF7431EDE0C0EA8225BA57DCD0C67D9EF5E948ED8' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/Users/xuzhichao/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:910: UserWarning: Ignoring URL 'https://weixin.sogou.com/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6cwJThYulHEtVjXrGTiVgS9dZesVPC425bopQRjCb2BlpQ4aYX0iWk1qXa8Fplpd9Hhup-WCiA6jKWLk-8yqbp2an3quTv2SP3gDmUfMBWk4hAl2leQHjMIc7a1X13FuuMHylJLJweSgqKYEhdkYntOId8_j3Lu1YmkUd6oG33m7wXpIptlAt7U6T7RdGxQgLBllaLKbmvInP_dLDClI7biNRhieJZTrlVKy4Eeemcj_f3q8uRDTfkw..&type=2&query=python&

In [ ]:
r.request

# 多个页面爬取

In [514]:
%%time
# C-1 多个页面+多个关键词
import time
from random import random

url = "https://weixin.sogou.com/weixin"
xpath_翻页a = '//div[@class="p-fy"]/a[starts-with(@href,"?query")]'
# 多界面（不同的内容（分类、关键词））多页面（翻页）
query = ['python','UX','李嘉琪','英雄联盟']
list_df = list()

## 第一页试探有多长的页面
for key in query:
    payload = 参数模板生成(query=[key], page=['1'])
    print(payload)
    df = request_sougou_weixin( url, params = payload)
    href_列表 = [x.xpath('//@href')[0] for x in r.html.xpath(xpath_翻页a)]
    df = pd.DataFrame([ urlparse(x) for x in href_列表])
    print(df)
    df_qs = pd.DataFrame([{k:v[0] for k,v in parse_qs(x).items()} for x in df['query']])
    df_qs = df_qs.assign (page_int=df_qs.page.astype(int)) # 变成整数
    长度 = df_qs.page_int.max()
    参数_query_X_page = { 
        i:参数模板生成(page = [i], \
                      query = [key]) \
        for i in range(df_qs.page_int.min()-1,长度)\
        }
    #print (参数_query_X_page)
    print (key,长度)
    print("参数_query_X_page",参数_query_X_page)
    
    for k,v in 参数_query_X_page.items():
        payload = v
        print("payload",payload)
        print(url)
        df = request_sougou_weixin( url, params = payload)
        
        time.sleep(3+4*random())  #放慢脚步 3-7秒, 平均约5秒
        # 备份
        display(df)
        df.to_csv("20春_Web数据挖掘_week05_sougou_weixin_{key}_{k}.tsv"\
                  .format(key=key, k=k), sep="\t", encoding="utf8")
        
        df = df.assign (query = key)  # 区分  keyword    
        df = df.assign (page = k)  # 区分  page    
        list_df.append(df)
        
df_all = pd.concat(list_df).reset_index()
df_all.index.name = '序'

df_all.to_excel("20春_Web数据挖掘_week05_sougou_weixin_翻页_5.xlsx",\
                sheet_name="_".join(query))
# 预估时间: 4*5秒*10 =200
# 预估数量: 4*40*10 =1600

{'query': ['python'], 'type': ['2'], 'page': ['1'], 'ie': ['utf8']}
  scheme netloc path params                                query fragment
0                             query=python&type=2&page=2&ie=utf8         
1                             query=python&type=2&page=3&ie=utf8         
2                             query=python&type=2&page=4&ie=utf8         
3                             query=python&type=2&page=5&ie=utf8         
4                             query=python&type=2&page=6&ie=utf8         
5                             query=python&type=2&page=7&ie=utf8         
6                             query=python&type=2&page=8&ie=utf8         
7                             query=python&type=2&page=9&ie=utf8         
8                            query=python&type=2&page=10&ie=utf8         
9                             query=python&type=2&page=2&ie=utf8         
python 10
参数_query_X_page {1: {'query': ['python'], 'type': ['2'], 'page': [1], 'ie': ['utf8']}, 2: {'query': ['python

,标题,摘要,source,时间,公众号链接,文章链接,格式化时间
0,手把手教你用 Python 绘制酷炫的桑基图!,"Python手把手绘制桑基图动手之前,我们再次敲黑板,回顾桑基图组成要素的重点——节点、边和...",CSDN,1585706777,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-01 02:06:17
1,Python 十大装 B 语法,作者 | 许向武责编 | 郭芮出品 | CSDN 博客Python 是一种代表简单思想的语言...,CSDN,1572764522,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2019-11-03 07:02:02
2,找对象那么难?python s3cmd 帮你解决,"python 的第三方库的安装目录:这个库实现 S3 协议的封装,解析,发包,收包等操作命令...",Python程序员,1586823024,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 00:10:24
3,7 款 Python 开源框架的优劣总结,下面就简单的介绍一下Python七款开源框架的优劣.DjangoDjango 应该是最出名的...,Python那些事,1586823354,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 00:15:54
4,为什么 Python 不是未来的编程语言?,"作者丨Rhea Moutafis译者丨刘志勇策划丨刘燕Python 现在如此火爆,已发展成为...",InfoQ,1586824246,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 00:30:46
5,Python绘制傅里叶变换、反变换与带通滤波图像,"推荐图书:《Python可以这样学》,ISBN:9787302456469,董付国,清华大学...",Python小屋,1586818537,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-13 22:55:37
6,MySQL与Python交互入门,"(0, ＂python1＂, 50), (0, ＂python2＂, 60), (0, ＂p...",小詹学Python,1586824285,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 00:31:25
7,python进阶-函数式编程(高级函数),"python sorted(iterable, key=None, reverse=Fals...",StriveNow,1586799046,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-13 17:30:46
8,用Python模拟新冠肺炎的传播,我想到的是单纯地在Python中使用可视化工具matplotlib来编写这个模拟过程.但是对...,Python程序员,1586823024,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 00:10:24
9,python_pytest基础知识1:,"1、python字典基础使用:dl = { ＂a＂ : None, ＂b＂ : 11, ＂c...",AdminWiki,1586797342,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-13 17:02:22


payload {'query': ['python'], 'type': ['2'], 'page': [2], 'ie': ['utf8']}
https://weixin.sogou.com/weixin


,标题,摘要,source,时间,公众号链接,文章链接,格式化时间
0,Python气象绘图教程(二),"Python有一个pip命令也能安装,但是我和某些气象家园的同学上过当,pip显示安装好了,...",云台书使,1586827009,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 01:16:49
1,Python 碾压 Excel,"根据最新消息,Python成了各大公司的心头好投行、券商、咨询、四大纷纷表示:想要顺利求职,...",证经学社,1586825623,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 00:53:43
2,Python中 Pandas 50题冲关,Python中的Numpy基础20问参考资料 | 100-pandas-puzzles - ...,Python那些事,1586823354,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-14 00:15:54
3,Python到底是什么?学姐靠它拿了5个offer,"Python的TIOBE指数持续上升,达到 8.530%,甚至超过C++的 7.384%,创...",遇见百种人生,1586790385,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-13 15:06:25
4,Python入门第八课:函数(1),"在VBA中,可以自定义函数(FUNCTION)过程,在Python中也一样,下面我们就来分享...",米宏Office,1586815219,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-13 22:00:19
5,Python小课教程:模块,"在Python中,一个.py文件就称之为一个模块(Module).使用模块有什么好处?最大的...",Python小课教程,1586793485,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-13 15:58:05
6,Python 抽象类原理以及编写,"Python 自带的abc模块用于实现抽象类相关的定义和操作.我们通过一个简单的例子来说明,...",菜鸟童靴,1586787412,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-13 14:16:52
7,"势头强劲的 Python PK 强大的 C++,究竟谁更胜一筹?","1.如果在Windows机器上用Windows编译器来编译代码,则会生成Windows机器代...",CSDN,1580544534,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-02-01 08:08:54
8,Python 什么时候会被取代?,"为什么Python不是未来的编程语言?作者 | Rhea Moutafis译者 | 弯月,责...",CSDN,1586052093,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-04-05 02:01:33
9,没错!Python杀死了Excel!,Python数据分析有多强大?举个栗子:用几行代码就能快速整理数据并出图:甚至可以编写合成多...,CSDN,1583654301,https://weixin.sogou.com/link?url=dn9a_-gY295K...,https://weixin.sogou.com/link?url=dn9a_-gY295K...,2020-03-08 07:58:21


payload {'query': ['python'], 'type': ['2'], 'page': [3], 'ie': ['utf8']}
https://weixin.sogou.com/weixin


...

,标题,摘要,source,时间,公众号链接,文章链接,格式化时间


payload {'query': ['英雄联盟'], 'type': ['2'], 'page': [9], 'ie': ['utf8']}
https://weixin.sogou.com/weixin


,标题,摘要,source,时间,公众号链接,文章链接,格式化时间


CPU times: user 1.27 s, sys: 114 ms, total: 1.38 s
Wall time: 3min 12s


/Users/xuzhichao/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:910: UserWarning: Ignoring URL 'https://weixin.sogou.com/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6EzDJysI4ql5MPrOUp16838dGRMI7NnPqR9qmSowG7r2gBNPyrh3FVgwvDqyjOWdzeOCKnRJAxkUdZw5uyu3Hynw3YU8OWoUNYcaOf58YCgqXH6IVD2Cttu-1hfDLPAhT5AG8baJOzSK67Mho2fdeLHDctAPZVNzluOsufU76d3ec5RqUeByHO-O00efWrWmm&type=2&query=python&token=210ED24AF7431EDE0C0EA8225BA57DCD0C67D9EF5E948ED8' with link or location/anchor > 255 characters since it exceeds Excel's limit for URLS
  force_unicode(url))
/Users/xuzhichao/anaconda3/lib/python3.7/site-packages/xlsxwriter/worksheet.py:910: UserWarning: Ignoring URL 'https://weixin.sogou.com/link?url=dn9a_-gY295K0Rci_xozVXfdMkSQTLW6cwJThYulHEtVjXrGTiVgS9dZesVPC425bopQRjCb2BlpQ4aYX0iWk1qXa8Fplpd9Hhup-WCiA6jKWLk-8yqbp2an3quTv2SP3gDmUfMBWk4hAl2leQHjMIc7a1X13FuuMHylJLJweSgqKYEhdkYntOId8_j3Lu1YmkUd6oG33m7wXpIptlAt7U6T7RdGxQgLBllaLKbmvInP_dLDClI7biNRhieJZTrlVKy4Eeemcj_f3q8uRDTfkw..&type=2&query=python&